In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
from torchtext import data
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import string
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using TensorFlow backend.


In [6]:

%matplotlib inline

In [7]:
def generate_trigrams(x):
    res = []
    n_grams = list(zip(*[x[i:] for i in range(3)]))
    for n_gram in n_grams:
        res.append(' '.join(n_gram))
    return res

In [13]:
import spacy
import en_core_web_sm

In [10]:
nlp = spacy.load("en_core_web_sm")

In [8]:
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
##############importing imdb daasets#############
TEXT = data.Field(preprocessing=generate_trigrams)
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT,LABEL)
print(vars(train_data.examples[0]))
train_data, valid_data = train_data.split(random_state=random.seed(SEED),split_ratio=0.9)
TEXT.build_vocab(train_data)
dictionary = dict(TEXT.vocab.freqs)

{'text': ["I don't know", "don't know who", 'know who could', 'who could find', 'could find fault', 'find fault with', 'fault with a', 'with a simply', 'a simply human', 'simply human and', 'human and funny', 'and funny film', 'funny film like', 'film like this', 'like this with', 'this with lots', 'with lots of', 'lots of delights', 'of delights for', 'delights for your', 'for your heart.', 'your heart. I', 'heart. I enjoyed', 'I enjoyed each', 'enjoyed each minute', 'each minute of', 'minute of it', 'of it and', 'it and guessed', 'and guessed the', 'guessed the ending', 'the ending half', 'ending half way', 'half way through', 'way through the', 'through the movie', 'the movie --', 'movie -- but', '-- but that', 'but that did', 'that did not', 'did not disappoint', 'not disappoint me', 'disappoint me at', 'me at all.', 'at all. It', 'all. It will', 'It will not', 'will not only', 'not only touch', 'only touch your', 'touch your heart', 'your heart but', "heart but it's", "but it's su

In [1]:
import torch
from torchtext import data, datasets
import nltk
import spacy
spacy.load('en')
import numpy as np
import random
import string
import re
from collections import Counter, defaultdict

In [20]:
# A few helper functions
def invert_dict(d):
    d_inv = defaultdict(list)
    for k, v in d.items():
        d_inv[v].append(k)
    return d_inv

In [2]:
# Markov n-gram doesn't need test data, so no need to split
TEXT = data.Field(lower=True, tokenize='spacy')
LABEL = data.LabelField(dtype = torch.float)
all_data = datasets.IMDB.splits(TEXT, LABEL, test=None)
all_data = all_data[0]

In [26]:
for ex in all_data.examples:
    for word in ex.text:
        word = re.sub(r'[^A-Za-z.!?\' ]', '', word)

In [27]:
# View sample review
print(f'Number of reviews: {len(all_data)}')
print(all_data.examples[0].text)

Number of reviews: 25000
['i', 'do', "n't", 'know', 'who', 'could', 'find', 'fault', 'with', 'a', 'simply', 'human', 'and', 'funny', 'film', 'like', 'this', 'with', 'lots', 'of', 'delights', 'for', 'your', 'heart', '.', 'i', 'enjoyed', 'each', 'minute', 'of', 'it', 'and', 'guessed', 'the', 'ending', 'half', 'way', 'through', 'the', 'movie', '--', 'but', 'that', 'did', 'not', 'disappoint', 'me', 'at', 'all', '.', 'it', 'will', 'not', 'only', 'touch', 'your', 'heart', 'but', 'it', "'s", 'such', 'a', 'good', 'family', 'friendly', 'film', '--', 'we', 'need', 'many', 'more', 'like', 'these', '!']


In [20]:
contractions = defaultdict(int)
for ex in all_data.examples:
    for word in ex.text:
        if word[0] == "'":
            contractions[word] += 1

In [21]:
contractions

defaultdict(int,
            {"'s": 62090,
             "'m": 4422,
             "'ll": 2703,
             "'": 17138,
             "'ve": 4946,
             "'re": 3653,
             "'d": 2212,
             "'<br": 29,
             "'cause": 55,
             "'mac": 1,
             "'---then": 1,
             "'---the": 1,
             "'em": 73,
             "'bout": 10,
             "'til": 1,
             "'big": 1,
             "''": 9,
             "'in": 1,
             "'coz": 2,
             "'cos": 5,
             "').<br": 1,
             "'the": 1,
             "'that": 1,
             "'till": 1,
             '\'"<br': 1,
             "'nuff": 2,
             "'do": 1,
             "'looks": 1,
             "'budget": 1})

In [28]:
contractions = defaultdict(int)
contr_prefix = defaultdict(list)
for ex in all_data.examples:
    for word in ex.text:
        if word[0] == "'":
            contractions[word.lefts()].append(word)

AttributeError: 'str' object has no attribute 'lefts'

In [22]:
# Create ngram dict from reviews
ngrams = defaultdict(list) # allows new keys to be added to ngrams with extra boilerplate
gram_size = 3

#words_tokens = nltk.word_tokenize(article_text)
for ex in all_data.examples:
    words_tokens = ex.text
    for i in range(len(words_tokens)-gram_size):
        seq = ' '.join(words_tokens[i:i+gram_size])
        #print(seq)
        #if  seq not in ngrams.keys():
        #    ngrams[seq] = []
        ngrams[seq].append(words_tokens[i+gram_size])

In [21]:
#def high_prob_word(ngram_key):
ngram_key = 'my favorite movie'
ngram_key = 'I liked the'
#for word in ngrams[ngram_key]:
freq_dict = invert_dict(Counter(ngrams[ngram_key]))
freq_keys = sorted(freq_dict.keys(), reverse=True)

        

In [29]:
freq_keys = sorted(freq_dict.keys(), reverse=True)

[8, 7, 6, 5, 4, 3, 2, 1]

In [79]:
def nextWord(words):
    freq_dict = invert_dict(Counter(words))
    freq_keys = sorted(freq_dict.keys(), reverse=True)
    possible_words = freq_dict[freq_keys[0]]
    next_word = possible_words[random.randrange(len(possible_words))]
    return next_word

In [11]:
# Check what's available as next words for our seed phrase
print(ngrams['my favorite movie'])

['at', '!', 'of', 'of', 'of', 'starring', 'genre', 'list', 'and', 'as', 'genre', 'almost', 'of', 'of', 'ever', 'he', 'ever', 'of', ',', 'but', 'of', 'and', ',', '!', '.', 'growing', '.', 'of', '.', '.', 'of', 'of']


In [25]:
curr_sequence = 'my favorite movie'
output = curr_sequence
#for i in range(17):
while len(output.split(sep=' ')) < 50:
    possible_words = ngrams[curr_sequence]
    next_word = possible_words[random.randrange(len(possible_words))]
    output += ' ' + next_word
    seq_words = nltk.word_tokenize(output)
    next_sequence = ' '.join(seq_words[len(seq_words)-gram_size:len(seq_words)])
    print(next_sequence)
    if next_sequence in ngrams.keys():
        prev_sequence = curr_sequence
        curr_sequence = next_sequence
    else:
        print('bad seq')
        curr_sequence = prev_sequence
        continue

print(len(output.split(sep=' ')), output)

favorite movie .
movie . what
. what 's
what 's more
's more ,
more , i
, i decided
i decided to
decided to give
to give ``
bad seq
give `` suck
bad seq
`` suck wait
bad seq
suck wait write
bad seq
wait write give
bad seq
write give check
bad seq
give check revisit
bad seq
check revisit watch
bad seq
revisit watch check
bad seq
watch check do
bad seq
check do go
bad seq
do go watch
bad seq
go watch rent
bad seq
watch rent revisit
bad seq
rent revisit continue
bad seq
revisit continue watch
bad seq
continue watch give
bad seq
watch give give
bad seq
give give watch
bad seq
give watch go
bad seq
watch go see
bad seq
go see never
bad seq
see never see
bad seq
never see stay
bad seq
see stay watch
bad seq
stay watch check
bad seq
watch check check
bad seq
check check pick
bad seq
check pick take
bad seq
pick take give
bad seq
take give watch
bad seq
give watch give
bad seq
watch give check
bad seq
give check go
bad seq
check go buy
bad seq
go buy take
bad seq
buy take put
bad seq
take put 

In [101]:
nltk.word_tokenize(output)

['my',
 'favorite',
 'movie',
 'he',
 'has',
 'ever',
 'been',
 'released',
 'on',
 'dvd',
 '.',
 'video',
 '.',
 'video',
 'video',
 'dvd',
 '.',
 'video',
 '.',
 'dvd',
 'dvd',
 '--',
 'this',
 'dvd',
 'dvd',
 '--',
 'this',
 'dvd']

In [ ]:
curr_sequence = 'my favorite movie'
output = curr_sequence
#for i in range(17):
while len(curr_sequence) < 20:
    if curr_sequence not in ngrams.keys():
        break
    possible_words = ngrams[curr_sequence]
    next_word = possible_words[random.randrange(len(possible_words))]
    #next_word = nextWord(possible_words)
    output += ' ' + next_word
    seq_words = nltk.word_tokenize(output)
    next_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])
    

print(output)